In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "/nfs/turbo/umms-vgvinodv/models/finetuned-checkpoints/radsum/biobart-large-mimic-cxr/checkpoint-19600"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.config.use_cache = False

model_ref = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [2]:
from datasets import load_dataset

dataset = load_dataset("varuUM/mimic-cxr-dpo-with-metrics", split="train")

In [3]:
def filter_func(examples):
    #return examples['rougeL'] < 0.3
    return examples['rougeL'] < 0.3 and examples['F1RadGraph'] < 0.3

dpo_dataset = dataset.filter(filter_func)
dpo_dataset = dpo_dataset.remove_columns(["rougeL","F1RadGraph","F1CheXbert"])

In [4]:
sanity_check = True
if sanity_check:
    num_samples = int(0.25*len(dpo_dataset)) #10000#
    dpo_dataset = dpo_dataset.select(range(num_samples))

print(len(dpo_dataset))
print(dpo_dataset[1])

7333
{'prompt': 'Heart size is normal. Mediastinal contours are normal with mild aortic tortuosity. Post-surgical changes in the right hemithorax are stable including thickening of the pleura along the costal surface and blunting of the costophrenic sulcus. The right sixth rib surgical fracture is redemonstrated. There are no new lung nodules identified. The main impression based on the given FINDINGS section of the chest X-ray report are:', 'chosen': 'Stable chest radiograph.', 'rejected': 'No radiographic evidence of pneumonia.'}


In [5]:
from transformers import TrainingArguments
from trl import DPOTrainer

dataset_config = "mimic-cxr"
model_name = "biobart-large"
save_path: str="/nfs/turbo/umms-vgvinodv/models/finetuned-checkpoints/radsum"
save_path = f"{save_path}/{model_name}-dpo-{dataset_config}"

training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    num_train_epochs = 1,
    max_steps =-1,
    save_strategy = 'epoch',
    save_total_limit = 1,
    logging_strategy ='steps',
    logging_steps=20,
    learning_rate=5.0e-7,
    output_dir=save_path,
    remove_unused_columns=False,
    run_name="dpo_biobart",
    overwrite_output_dir = True,
    bf16=True,
)


dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    max_prompt_length=512,
    max_length=1024,
)

In [6]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
20,1.775900


TrainOutput(global_step=28, training_loss=1.6712019443511963, metrics={'train_runtime': 922.5159, 'train_samples_per_second': 7.949, 'train_steps_per_second': 0.03, 'total_flos': 0.0, 'train_loss': 1.6712019443511963, 'epoch': 0.98})

In [7]:
model.to("cuda")

prompt = "The main impression based on the given FINDINGS section of the chest X-ray report are:"
findings_example = f"""Lateral view somewhat limited due to overlying motion artifact. The lungs are low in volume.  There is no focal airspace consolidation to suggest pneumonia.  A 1.2-cm calcified granuloma just below the medial aspect of the right hemidiaphragm is unchanged from prior study.  No pleural effusions or pulmonary edema. There is no pneumothorax.  The inferior sternotomy wire is fractured but unchanged. Surgical clips and vascular markers in the thorax are related to prior CABG surgery."""
eval_prompt = findings_example + prompt
print(f"Model Input:\n{eval_prompt}\n")

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print("---\nGenerated Output:\n")
    print(tokenizer.decode(model.generate(**model_input, max_length=256)[0], skip_special_tokens=True).split(":")[-1].strip())
    
ground_truth_summary="""
---
Ground Truth:
No evidence of acute cardiopulmonary process.
"""    
print(ground_truth_summary)

Model Input:
Lateral view somewhat limited due to overlying motion artifact. The lungs are low in volume.  There is no focal airspace consolidation to suggest pneumonia.  A 1.2-cm calcified granuloma just below the medial aspect of the right hemidiaphragm is unchanged from prior study.  No pleural effusions or pulmonary edema. There is no pneumothorax.  The inferior sternotomy wire is fractured but unchanged. Surgical clips and vascular markers in the thorax are related to prior CABG surgery.The main impression based on the given FINDINGS section of the chest X-ray report are:

---
Generated Output:

No acute cardiopulmonary process.

---
Ground Truth:
No evidence of acute cardiopulmonary process.



In [11]:
from nltk.tokenize import wordpunct_tokenize
from radgraph import F1RadGraph
from f1chexbert import F1CheXbert
import datasets
from pathlib import Path

def build_test_dataset(dataset_config, tokenizer, split="test"):
    data_path = '/nfs/turbo/umms-vgvinodv/data/bioNLP23-Task-1B/data/'
    findings_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.findings.tok')
    impression_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.impression.tok')


    findings = [line.strip() for line in open(findings_file_path).readlines()]
    impression = [line.strip() for line in open(impression_file_path).readlines()]

    dataset = datasets.Dataset.from_dict({"text":findings,"summary":impression}) 
    
    return dataset

def generate_summary(sample):
    texts = sample["text"]
    summaries = sample["summary"]
    prompt = "The main impression based on the given FINDINGS section of the chest X-ray report are:"

    def generate_input(_text):
        return " ".join([_text,prompt])

    inputs = generate_input(texts) 
    model_input = tokenizer(inputs, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True)
    
    formatted_response = response.split(":")[-1].strip()
    return {
        "text": inputs,
        "summary":summaries,
        "pred": formatted_response
    }

def process_impression(impression):
    impression = impression.lower()
    return ' '.join(wordpunct_tokenize(impression))

In [12]:
mini_test = False

test_dataset = build_test_dataset('mimic-cxr',tokenizer,'test')

if mini_test:
    num_samples = int(0.25*len(test_dataset))
    test_dataset = test_dataset.select(range(num_samples))

print(f'Number of test samples: {len(test_dataset)}')

Number of test samples: 1624


In [13]:
model.eval()
model.to("cuda")
results = test_dataset.map(generate_summary, remove_columns=list(test_dataset.features))

pred_str = results["pred"]
pred_str = list(map(process_impression,pred_str))
label_str = results["summary"]

import numpy as np
import evaluate

###################################
rouge = datasets.load_metric("rouge")
rouge_output = rouge.compute(predictions=pred_str, references=label_str)

res = {key: value.mid.fmeasure * 100 for key, value in rouge_output.items()}
print('ROUGE:')
print({k: round(v, 4) for k, v in res.items()})

#################################
f1radgraph = F1RadGraph(reward_level="partial")
score = f1radgraph(hyps=pred_str,refs=label_str)[0]
print("\nF1RadGraph:")
print(score*100)

#################################
f1chexbert = F1CheXbert(device="cuda")
accuracy, accuracy_not_averaged, class_report, class_report_5 = f1chexbert(
    hyps=pred_str,
    refs=label_str)
print("\nF1CheXbert:")
print(class_report_5["micro avg"]["f1-score"])

Parameter 'function'=<function generate_summary at 0x154a9fcf3e20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1624 [00:00<?, ? examples/s]

ROUGE:
{'rouge1': 49.6307, 'rouge2': 34.7811, 'rougeL': 45.8691, 'rougeLsum': 45.8348}

F1RadGraph:
42.725627975816636

F1CheXbert:
0.7363636363636362


In [14]:
import numpy as np
import evaluate

hidden_test_dataset = build_test_dataset('mimic-cxr',tokenizer,'test.hidden')

print(f'Number of Hidden test samples: {len(hidden_test_dataset)}\n')

results_hidden_test = hidden_test_dataset.map(generate_summary, remove_columns=list(hidden_test_dataset.features))

pred_str = results_hidden_test["pred"]
pred_str = list(map(process_impression,pred_str))
label_str = results_hidden_test["summary"]

###################################
rouge = datasets.load_metric("rouge")
rouge_output = rouge.compute(predictions=pred_str, references=label_str)

res = {key: value.mid.fmeasure * 100 for key, value in rouge_output.items()}
print('ROUGE:')
print({k: round(v, 4) for k, v in res.items()})

#################################
f1radgraph = F1RadGraph(reward_level="partial")
score = f1radgraph(hyps=pred_str,refs=label_str)[0]
print("\nF1RadGraph:")
print(score*100)

#################################
f1chexbert = F1CheXbert(device="cuda")
accuracy, accuracy_not_averaged, class_report, class_report_5 = f1chexbert(
    hyps=pred_str,
    refs=label_str)
print("\nF1CheXbert:")
print(class_report_5["micro avg"]["f1-score"])

Number of Hidden test samples: 1000



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

ROUGE:
{'rouge1': 36.2777, 'rouge2': 21.7235, 'rougeL': 31.3638, 'rougeLsum': 31.3745}

F1RadGraph:
13.400771126338892

F1CheXbert:
0.6236335242061426


# Testing on small batch

In [8]:
from radgraph import F1RadGraph
import evaluate
import pandas as pd

rouge = evaluate.load('rouge')
f1radgraph = F1RadGraph(reward_level="partial")

generation_kwargs = {
#    "min_length": -1,
#    "top_k": 0.0,
#    "top_p": 1.0,
#    "do_sample": True,
    "max_new_tokens": 512,
}

bs = 32
game_data = dict()
filtered_dataset = dpo_dataset
filtered_dataset.set_format("pandas")
df_batch = filtered_dataset[:].sample(bs)

game_data["GT"] = df_batch["chosen"].tolist()

query_tensors = tokenizer(df_batch["prompt"].tolist()).input_ids
response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    output = model_ref.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to("cuda"), **generation_kwargs).squeeze()
    response_tensors_ref.append(output)
    output = model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to("cuda"), **generation_kwargs).squeeze()
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i],skip_special_tokens=True).split(":")[-1].strip() for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i],skip_special_tokens=True).split(":")[-1].strip() for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
game_data["F1RadGraph rewards (before)"] = f1radgraph(hyps=game_data["response (before)"], refs=game_data["GT"])[1]
game_data["F1RadGraph rewards (after)"] = f1radgraph(hyps=game_data["response (after)"], refs=game_data["GT"])[1]

game_data["rougeL rewards (before)"] = rouge.compute(predictions=game_data["response (before)"], references=game_data["GT"], rouge_types=['rougeL'],  use_aggregator=False)['rougeL']
game_data["rougeL rewards (after)"] = rouge.compute(predictions=game_data["response (after)"], references=game_data["GT"], rouge_types=['rougeL'],  use_aggregator=False)['rougeL']

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,GT,response (before),response (after),F1RadGraph rewards (before),F1RadGraph rewards (after),rougeL rewards (before),rougeL rewards (after)
0,No opacity convincing for pneumonia.,No evidence of acute intrathoracic process.,No evidence of acute intrathoracic process.,0.000000,0.000000,0.181818,0.181818
1,No evidence of pneumonia.,No acute cardiopulmonary process.,No acute cardiopulmonary process.,0.000000,0.000000,0.250000,0.250000
2,Mild right basal atelectasis. No convincing ev...,Mild right basal atelectasis.,"Right basal atelectasis, otherwise unremarkable.",0.666667,0.400000,0.615385,0.428571
3,"1. Chronic pulmonary changes, predominately in...",1. No evidence of pneumonia. 2. Prominence of ...,1. No evidence of pneumonia. 2. Stable promine...,0.142857,0.133333,0.214286,0.206897
4,Findings concerning for small bowel obstructio...,1. No free air below the right hemidiaphragm. ...,1. No free air below the right hemidiaphragm. ...,0.434783,0.434783,0.444444,0.444444
5,Relatively low lung volumes without focal cons...,No acute cardiopulmonary process.,No acute cardiopulmonary process.,0.000000,0.000000,0.000000,0.000000
6,Normal chest x-ray.,No acute cardiopulmonary process.,No acute cardiopulmonary process.,0.000000,0.000000,0.000000,0.000000
7,Interval development of fluid overload and mil...,Interval development of central pulmonary vasc...,Interval development of central pulmonary vasc...,0.500000,0.500000,0.333333,0.333333
8,No definite focal consolidation to suggest pne...,No acute intrathoracic process.,No acute intrathoracic process.,0.000000,0.000000,0.181818,0.181818
9,No lung consolidation or pulmonary edema.,No acute cardiopulmonary process.,No acute cardiopulmonary process.,0.000000,0.000000,0.200000,0.200000


In [9]:
from datasets  import Dataset

result = Dataset.from_pandas(df_results)

In [10]:
result[5]

{'GT': 'Relatively low lung volumes without focal consolidation.',
 'response (before)': 'No acute cardiopulmonary process.',
 'response (after)': 'No acute cardiopulmonary process.',
 'F1RadGraph rewards (before)': 0.0,
 'F1RadGraph rewards (after)': 0.0,
 'rougeL rewards (before)': 0.0,
 'rougeL rewards (after)': 0.0}